# Data Preprocessing

    1. Importing the libraries
    2. Importing the Dataset
    3. Handling of Missing Data
    4. Handling of Categorical Data
    5. Splitting the dataset into training and testing datasets
    6. Feature Scaling

In [17]:
# Importing libraries
import sklearn
import pandas as pd
import numpy as np
import re

In [18]:
# Importing dataset
path = '/content/drive/MyDrive/Reading/[2]Machine_learning/Hands-on-Machine-Learning-master/datasets/housing/housing.csv'
df = pd.read_csv(path)
df

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,INLAND


In [19]:
# Handling missing values

'''
isna(): https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.isna.html
This function returns a *dataframe* that each cell stores a boolean value (True/False) stating whether this cell has a missing value.
True: The value of this cell is missing
False: The value of this cell is not missing
'''
# sum(): https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sum.html

number_of_missing_values_in_each_column = df.isna().sum()
number_of_missing_values_in_each_column

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

We have 207 missing values on the column 'total_bedrooms'.
There are many ways to handle missing values, e.g. dropping the rows that contains missing values, filling in the cell with median/mean/mode of the column.

1. Deleting the rows with missing values
There is a function called [`dropna()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html) that will help delete the row/column with missing values. 
    - Only delete rows with missing values when the number of missing values of a column is small.
    - We define `axis` parameters to decide whether this function will delete rows or columns with missing values.
      * 0, or ‘index’ : Drop rows which contain missing values.
      * 1, or ‘columns’ : Drop columns which contain missing value.

2. Imputation (filling in the cell with median/mean/mode of the column)
    1. If the missing value is numerical (numbers)
      - Fill in mean/median of the column
    2. If the missing values is categorical (string)
      - Fill in mode of the column (Mode: the most frequent value)

In [20]:
# To delete the rows with missing values (Not recommended for this dataset!)
new_df = df.dropna(axis=0)
new_df.isna().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
ocean_proximity       0
dtype: int64

In [21]:
'''
Column 'total_bedrooms' contains numerical values --> We fill in missing value with either mean or median.
First, we can use `df.fillna()` function to complete missing values. This needs to specify the column name.
'''
# Use mean to fill in
# df.mean(): https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mean.html
new_df_mean = df
new_df_mean['total_bedrooms'] = new_df_mean['total_bedrooms'].fillna(new_df_mean['total_bedrooms'].mean())
new_df_mean.isna().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
ocean_proximity       0
dtype: int64

In [22]:
# Use median to fill in 
# df.median(): https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.median.html
new_df_median = df
new_df_median['total_bedrooms'] = new_df_median['total_bedrooms'].fillna(new_df_median['total_bedrooms'].median())
new_df_median.isna().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
ocean_proximity       0
dtype: int64

Second, we can also use [SimpleImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) to impute (to complete the missing values).

Since we have some non-numerical (categorical) values (Column `'ocean_proximity'`) and we cannot use mean to complete categorical value,
we have to deal with categorical column first.

To handle categorical data, basically we tranform strings into numbers so the computer can recognize them. The tranformation is simple:
- Category 1 --> 1
- Category 2 --> 2
- Category 3 --> 3
...

We can use [LabelEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html) from sklearn to finish the transformation.

In [23]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['ocean_proximity'] = encoder.fit_transform(df['ocean_proximity'])
df['ocean_proximity']

0        3
1        3
2        3
3        3
4        3
        ..
20635    1
20636    1
20637    1
20638    1
20639    1
Name: ocean_proximity, Length: 20640, dtype: int64

In [24]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(fill_value=np.nan, strategy='mean')
X = imputer.fit_transform(df)    # Returns an array
df = pd.DataFrame(X, columns=df.columns)
df.isna().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
ocean_proximity       0
dtype: int64

`df.fillna()` is more preferrable for handling missing values as it doesn't need to deal with columns with categorical data.

After handling the missing values, we have to handle categorical data. As we already transform the categorical column into numbers, we'll skip this step.

To split the training and testing datasets, the most common way is to use sklearn's function [`model_selection.train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

Usually the ratio of train and test dataset will be 80:20.

Need to remember the return sequence (first two is X, next two is y).
Reminded that y is the answer.

In [26]:
from sklearn.model_selection import train_test_split
Y = df['median_house_value']
X = df.drop(columns='median_house_value')

X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.8) 

print(X_train)
print(X_test)
print(y_train)
print(y_test)

       longitude  latitude  ...  median_income  ocean_proximity
17456    -120.43     34.69  ...         3.6023              4.0
13880    -117.34     34.46  ...         3.1505              1.0
13761    -117.09     34.01  ...         4.0556              1.0
15896    -122.39     37.72  ...         1.1125              3.0
3631     -118.43     34.23  ...         4.4432              0.0
...          ...       ...  ...            ...              ...
13844    -117.33     34.53  ...         3.0943              1.0
3964     -118.58     34.20  ...         3.5000              0.0
20603    -121.54     39.13  ...         1.3375              1.0
15531    -117.12     33.15  ...         4.4655              0.0
14783    -117.09     32.56  ...         4.8984              4.0

[16512 rows x 9 columns]
       longitude  latitude  ...  median_income  ocean_proximity
1842     -122.29     37.91  ...         5.5357              3.0
1771     -122.35     37.95  ...         3.6094              3.0
1034     -120.